In [ ]:
!pip install pingouin

import pandas as pd
import pingouin as pg
import numpy as np

def calculate_icc(file_path):
    data = pd.read_csv(file_path)
    return pg.intraclass_corr(data=data, targets='targets', raters='raters', ratings='ratings'), data

def calculate_sems_and_mdcs(icc_value, data, suffix):
    s = np.std(data['ratings'], ddof=1)
    sem = s * np.sqrt(1 - icc_value)
    ratings_diff = data.groupby('targets')['ratings'].diff().dropna()
    sd_diff = np.std(ratings_diff, ddof=1)
    sem_diff = sd_diff / np.sqrt(2)

    mdc95 = sem * 1.96 * np.sqrt(2)
    mdc95_diff = sem_diff * 1.96 * np.sqrt(2)

    print(f"SEM (using ICC): {sem:.4f}")
    print(f"SEM (using SD Diff): {sem_diff:.4f}")
    print(f"MDC95 (using ICC): {mdc95:.4f}")
    print(f"MDC95 (using SD Diff): {mdc95_diff:.4f}")

def main_icc(file_path):
    icc_results, data = calculate_icc(file_path)

    for icc_type in ['ICC1', 'ICC2', 'ICC3']:
        icc_row = icc_results[icc_results['Type'] == icc_type]
        if not icc_row.empty:
            icc_value = icc_row['ICC'].values[0]
            p_value = icc_row['pval'].values[0]
            ci95 = icc_row['CI95%'].values[0]

            print(f"\n========= Results for {icc_type} =========")  
            print(f"{icc_type}: {icc_value:.4f}")
            print(f"{icc_type}のp値: {p_value:.4e}")  
            print(f"{icc_type}の95%信頼区間: {ci95}")

            calculate_sems_and_mdcs(icc_value, data, icc_type[-1])

# 以下の「hoge.csv」の場所にて、CSVファイルの名前を書き換えてください。
main_icc('hoge.csv')
